In [1]:
import numpy as np
import scipy
import itertools
import matplotlib.pyplot as plt
rng = np.random.default_rng(seed=0)

# Introduction

In today's notebook, we'll be demonstrating the exact solution of 2D Yang-Mills theories. As we saw in the lecture, Wilson loops should exactly decay exponentially in the area of the Wilson loop. There are "no excited state contributions". In fact, it doesn't matter what the actual shape of the Wilson loop is: in the plot below, there are two examples shown (1xN rectangles, and L-shaped tetris-tiles). 
![example](LGT1-Unit02-Day10_2d_example.png)

Your task is to reproduce this plot for the 1xN shaped rectangles. 

<b>Challenge:</b> Implement a different type of shape and check that it still lies on the same curve (e.g. L-shaped loops, or something stranger). 

In [2]:
Nc            = 2             #Number of colors
Nd            = 2             #Number of spacetime dimensions
L             = 20            #Box length
beta          = 6             #Action = (beta/Nc) Sum_{unoriented plaquettes} ReTr(P)
lattice_shape = tuple([L]*Nd) #Shape of the lattice

Here are some auxiliary functions that we've seen from last week's notebook, that will let us sample the path integral with the heatbath algorithm. 

In [3]:
def trace(M):
    # Trace over the last two dimension
    return np.trace(M, axis1=-1, axis2=-2)

def adjoint(M):
    #Adjoint (dagger) of the color matrix
    return np.conj(np.swapaxes(M, axis1=-1, axis2=-2))

def project_suN(g):
    #Projects an array of matrices g onto SU(N)
    def project_uN(g):
        Nc = g.shape[-1]
        for i in range(Nc):
            v_i = g[..., i, :]
            for j in range(i):
                v_j = g[..., j, :]
                dot = np.sum(np.conj(v_j) * v_i, axis=-1)
                v_i = v_i - dot[..., None] * v_j
            v_i = v_i / np.linalg.norm(v_i, axis=-1)[..., None]
            g[..., i, :] = v_i
        return g
    Nc = g.shape[-1]
    g = project_uN(g)
    return g * np.conj(np.linalg.det(g)**(1/Nc))[...,None,None]

def compute_staple(U, mu):
    #Computes the staple, required for heatbath
    result = 0
    for nu in range(Nd):
        if nu == mu:
            continue
        staple_left = np.roll(U[nu], -1, mu) @ np.roll(adjoint(U[mu]), -1, nu) @ adjoint(U[nu])
        staple_right = np.roll(adjoint(U[nu]), (-1, 1), (mu, nu)) @ np.roll(adjoint(U[mu]), 1, nu) @ np.roll(U[nu], 1, nu)

        result = result + staple_left + staple_right
    return result

def su2_heatbath(U, mu, *, beta, update_mask):
    #Performs an update of the gauge field in the mu dimension
    def _sample_x0(a):
        accepted = np.zeros(a.shape, dtype=bool)
        x0 = np.zeros_like(a)
        while np.sum(accepted) < np.sum(np.ones_like(a)):
            def _uniform(maxval=1.0):
                return rng.uniform(low=0.0, high=maxval, size=a.shape)

            r = _uniform()
            r1 = 1.0 - _uniform()
            r2 = _uniform(maxval=2*np.pi)
            r3 = 1.0 - _uniform()

            lam_sq = -1 / (4 * a) * (np.log(r1) + np.cos(r2)**2 * np.log(r3))

            newly_accepted = r**2 < 1 - lam_sq
            to_update = np.logical_and(newly_accepted, np.logical_not(accepted))

            x0 = np.where(to_update, 1 - 2 * lam_sq, x0)

            accepted = np.logical_or(accepted, newly_accepted)

        return x0
    
    def _su2_heatbath_dist(a):
        x0 = _sample_x0(a)
        r = rng.normal(size=(3, *a.shape))
        x = r / np.linalg.norm(r, axis=0) * np.sqrt(1 - x0**2)
        X = np.zeros((*a.shape, 2, 2), dtype=np.complex128)
        # x0 + i x[2] sigma_z                                                                                                                                                   
        X[..., 0, 0] = x0 + 1j * x[2]
        X[..., 1, 1] = x0 - 1j * x[2]
        # i x[0] sigma_x + i x[1] sigma_y                                                                                                                                       
        X[..., 0, 1] = 1j * x[0] + x[1]
        X[..., 1, 0] = 1j * x[0] - x[1]
        return X
    
    A = beta / Nc * compute_staple(U, mu)
    a = np.sqrt(np.abs(np.linalg.det(A)))
    X = _su2_heatbath_dist(a)
    U_mu_new = X @ adjoint(A) / a[..., None, None]
    U[mu] = U[mu] * update_mask[...,None,None] + (1 - update_mask[...,None,None]) * U_mu_new
    U = project_suN(U)
    return U

def make_even_mask(lattice_shape):
    #Auxiliary function to make an even/odd mask, for heatbath
    even_mask = np.zeros(lattice_shape)
    for parities in itertools.product([0, 1], repeat=Nd):
        idx = tuple(slice(parity, None, 2) for parity in parities)
        if sum(parities) % 2 == 0:
            even_mask[idx] = 1
    return even_mask

def heatbath_sweep(U, beta):
    #Perform a heatbath sweep over the whole lattice
    even_mask = make_even_mask(lattice_shape=U.shape[1:-2])

    for mu in range(Nd):
        U = su2_heatbath(U, mu, beta=beta, update_mask=even_mask)
        U = su2_heatbath(U, mu, beta=beta, update_mask=1-even_mask)
    return U

The following function will be useful for you to use, as it will compute averaged Wilson loops on your generated gauge configurations:

In [4]:
def wilson_loop(U, mu, nu, l_mu, l_nu, untraced=False):
    """
    Computes the average of the trace of the Wilson loop in the mu-th and nu-th directions.
    The Wilson loop has length l_mu in the mu-th direction, and l_nu in the nu-th direction.
    If untraced is true, then the unaveraged, untraced Wilson loop is returned. 
    """
    W = np.broadcast_to(np.eye(Nc, Nc), U.shape[1:])
    for x in range(l_mu):
        W = W @ np.roll(U[mu], -x, mu)
    for y in range(l_nu):
        W = W @ np.roll(U[nu], (-l_mu, -y), (mu, nu))
    for x in reversed(range(l_mu)):
        W = W @ np.roll(adjoint(U[mu]), (-x, -l_nu), (mu, nu))
    for y in reversed(range(l_nu)):
        W = W @ np.roll(adjoint(U[nu]), -y, nu)

    if untraced:
        return W
    
    return np.mean(trace(W.real))

# Your Code Below:

In [5]:
loops = [];
U = project_suN(   np.random.random((Nd,*lattice_shape,Nc,Nc)) + \
                1j*np.random.random((Nd,*lattice_shape,Nc,Nc)))

In [7]:
def compute_loops(U):
    #??
    pass

In [8]:
for _ in range(20):
    U = heatbath_sweep(U, beta)
for i in range(100):
    for _ in range(10):
        U = heatbath_sweep(U, beta)
    #loops.append(compute_loops(U))
    if i%10 == 0:
        print(f"Iter {i}/100")

Iter 0/100
Iter 10/100
Iter 20/100
Iter 30/100
Iter 40/100
Iter 50/100
Iter 60/100
Iter 70/100
Iter 80/100
Iter 90/100
